# **WNBA Prediction**

In [6]:
# install packages
%pip install pandas
%pip install scikit-learn

# **Part 1: Cleaning Data**

In [7]:
# import pandas
import pandas as pd

In [8]:
# read schedule CSV file and remove unecessary columns
schedule = pd.read_csv("reg_season.csv")

In [ ]:
schedule.head()

,Date,Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 5,Notes
0,Fri May 19 2023,Connecticut Sun,70,Indiana Fever,61,Box Score,Commissioner's Cup Game
1,Fri May 19 2023,Phoenix Mercury,71,Los Angeles Sparks,94,Box Score,Commissioner's Cup Game
2,Fri May 19 2023,Chicago Sky,77,Minnesota Lynx,66,Box Score,NaN
3,Fri May 19 2023,New York Liberty,64,Washington Mystics,80,Box Score,Commissioner's Cup Game
4,Sat May 20 2023,Atlanta Dream,78,Dallas Wings,85,Box Score,NaN


In [9]:
schedule = schedule.iloc[:, 1:-2]

In [10]:
schedule.head()

,Visitor/Neutral,PTS,Home/Neutral,PTS.1
0,Connecticut Sun,70,Indiana Fever,61
1,Phoenix Mercury,71,Los Angeles Sparks,94
2,Chicago Sky,77,Minnesota Lynx,66
3,New York Liberty,64,Washington Mystics,80
4,Atlanta Dream,78,Dallas Wings,85


In [11]:
advanced_stats = pd.read_csv("advanced_stats.csv")

In [12]:
# Remove nan columns
advanced_stats = advanced_stats.dropna(axis=1, how='all')
# Remove fist and last columns (rank, and arena)
advanced_stats = advanced_stats.iloc[:, 1:-1]
# Display dataframe
advanced_stats.head()

,Team,Age,W,L,PW,PL,MOV,SOS,SRS,ORtg,...,3PAr,TS%,eFG%,TOV%,ORB%,FT/FGA,eFG%.1,TOV%.1,DRB%,FT/FGA.1
0,Las Vegas Aces,29.0,34.0,6.0,35,5,12.55,-1.09,11.46,114.8,...,0.360,0.597,0.553,12.5,18.9,0.236,0.480,14.4,78.2,0.171
1,New York Liberty,28.8,32.0,8.0,32,8,8.68,-0.53,8.14,111.8,...,0.424,0.577,0.539,14.9,25.8,0.198,0.476,13.1,78.5,0.184
2,Connecticut Sun,29.1,27.0,13.0,26,14,3.70,-0.27,3.43,105.4,...,0.294,0.540,0.498,14.0,23.9,0.224,0.486,16.7,75.8,0.236
3,Dallas Wings,26.0,22.0,18.0,25,15,2.95,-0.31,2.64,109.5,...,0.292,0.535,0.489,13.7,32.8,0.224,0.504,15.0,77.2,0.237
4,Washington Mystics,28.8,19.0,21.0,19,21,-0.35,0.03,-0.32,101.5,...,0.342,0.533,0.486,13.9,18.7,0.222,0.496,16.3,78.0,0.238


In [17]:
df = pd.merge(schedule, advanced_stats, left_on="Visitor/Neutral", right_on="Team")
df = pd.merge(df, advanced_stats, left_on="Home/Neutral", right_on="Team")

In [18]:
df = df.drop(["Team_x", "Team_y"], axis=1)

In [19]:
df.head()

,Visitor/Neutral,PTS,Home/Neutral,PTS.1,Age_x,W_x,L_x,PW_x,PL_x,MOV_x,...,3PAr_y,TS%_y,eFG%_y,TOV%_y,ORB%_y,FT/FGA_y,eFG%.1_y,TOV%.1_y,DRB%_y,FT/FGA.1_y
0,Connecticut Sun,70,Indiana Fever,61,29.1,27.0,13.0,26,14,3.70,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248
1,Connecticut Sun,88,Indiana Fever,72,29.1,27.0,13.0,26,14,3.70,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248
2,Phoenix Mercury,85,Indiana Fever,82,27.8,9.0,31.0,8,32,-8.30,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248
3,Phoenix Mercury,71,Indiana Fever,72,27.8,9.0,31.0,8,32,-8.30,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248
4,Chicago Sky,89,Indiana Fever,87,27.2,18.0,22.0,17,23,-1.63,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248


In [20]:
# Add a column for to show if the home team won
for index, row in df.iterrows():
    # Determine which team had more points
    if df.loc[index, 'PTS'] > df.loc[index, 'PTS.1']:
        # Place 0 for home loss
        df.loc[index, 'Home_Winner'] = 0
    else:
        # Place 1 for home win
        df.loc[index, 'Home_Winner'] = 1

# Display dataframe
df.head()

,Visitor/Neutral,PTS,Home/Neutral,PTS.1,Age_x,W_x,L_x,PW_x,PL_x,MOV_x,...,TS%_y,eFG%_y,TOV%_y,ORB%_y,FT/FGA_y,eFG%.1_y,TOV%.1_y,DRB%_y,FT/FGA.1_y,Home_Winner
0,Connecticut Sun,70,Indiana Fever,61,29.1,27.0,13.0,26,14,3.70,...,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248,0.0
1,Connecticut Sun,88,Indiana Fever,72,29.1,27.0,13.0,26,14,3.70,...,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248,0.0
2,Phoenix Mercury,85,Indiana Fever,82,27.8,9.0,31.0,8,32,-8.30,...,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248,0.0
3,Phoenix Mercury,71,Indiana Fever,72,27.8,9.0,31.0,8,32,-8.30,...,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248,1.0
4,Chicago Sky,89,Indiana Fever,87,27.2,18.0,22.0,17,23,-1.63,...,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248,0.0


In [21]:
remove_cols = ["Home/Neutral", "Visitor/Neutral", "PTS", "PTS.1", "Home_Winner"]
selected_cols = [x for x in df.columns if x not in remove_cols]
df[selected_cols].head()

,Age_x,W_x,L_x,PW_x,PL_x,MOV_x,SOS_x,SRS_x,ORtg_x,DRtg_x,...,3PAr_y,TS%_y,eFG%_y,TOV%_y,ORB%_y,FT/FGA_y,eFG%.1_y,TOV%.1_y,DRB%_y,FT/FGA.1_y
0,29.1,27.0,13.0,26,14,3.70,-0.27,3.43,105.4,100.7,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248
1,29.1,27.0,13.0,26,14,3.70,-0.27,3.43,105.4,100.7,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248
2,27.8,9.0,31.0,8,32,-8.30,0.55,-7.75,99.1,109.9,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248
3,27.8,9.0,31.0,8,32,-8.30,0.55,-7.75,99.1,109.9,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248
4,27.2,18.0,22.0,17,23,-1.63,0.11,-1.52,103.3,105.3,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248


In [23]:
from sklearn.preprocessing import MinMaxScaler

In [24]:
# Initialize scaler
scalar = MinMaxScaler()
# Scale our data
df[selected_cols] = scalar.fit_transform(df[selected_cols])
# Display dataframe
df.head()

,Visitor/Neutral,PTS,Home/Neutral,PTS.1,Age_x,W_x,L_x,PW_x,PL_x,MOV_x,...,TS%_y,eFG%_y,TOV%_y,ORB%_y,FT/FGA_y,eFG%.1_y,TOV%.1_y,DRB%_y,FT/FGA.1_y,Home_Winner
0,Connecticut Sun,70,Indiana Fever,61,1.00000,0.72,0.28,0.666667,0.333333,0.575540,...,0.177215,0.225,0.659574,0.546099,0.392405,0.891892,0.216216,0.690909,0.9625,0.0
1,Connecticut Sun,88,Indiana Fever,72,1.00000,0.72,0.28,0.666667,0.333333,0.575540,...,0.177215,0.225,0.659574,0.546099,0.392405,0.891892,0.216216,0.690909,0.9625,0.0
2,Phoenix Mercury,85,Indiana Fever,82,0.59375,0.00,1.00,0.000000,1.000000,0.000000,...,0.177215,0.225,0.659574,0.546099,0.392405,0.891892,0.216216,0.690909,0.9625,0.0
3,Phoenix Mercury,71,Indiana Fever,72,0.59375,0.00,1.00,0.000000,1.000000,0.000000,...,0.177215,0.225,0.659574,0.546099,0.392405,0.891892,0.216216,0.690909,0.9625,1.0
4,Chicago Sky,89,Indiana Fever,87,0.40625,0.36,0.64,0.333333,0.666667,0.319904,...,0.177215,0.225,0.659574,0.546099,0.392405,0.891892,0.216216,0.690909,0.9625,0.0


# **Part 2: Determine Predictors**

In [25]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

# Initialize our ridge regression classification
rr = RidgeClassifier(alpha=1.0)

# Initialize our feature selector which picks the best 10 features backward
sfs = SequentialFeatureSelector(rr, n_features_to_select=10, direction='backward')

In [26]:
sfs.fit(df[selected_cols], df['Home_Winner'])

SequentialFeatureSelector(direction='backward', estimator=RidgeClassifier(),
                          n_features_to_select=10)

In [27]:
predictors = list(df[selected_cols].columns[sfs.get_support()])

In [28]:
df[predictors].head()

,ORB%_x,eFG%.1_x,TOV%.1_x,L_y,TS%_y,eFG%_y,ORB%_y,FT/FGA_y,eFG%.1_y,TOV%.1_y
0,0.368794,0.270270,0.972973,0.84,0.177215,0.225,0.546099,0.392405,0.891892,0.216216
1,0.368794,0.270270,0.972973,0.84,0.177215,0.225,0.546099,0.392405,0.891892,0.216216
2,0.156028,0.810811,0.081081,0.84,0.177215,0.225,0.546099,0.392405,0.891892,0.216216
3,0.156028,0.810811,0.081081,0.84,0.177215,0.225,0.546099,0.392405,0.891892,0.216216
4,0.404255,0.621622,0.405405,0.84,0.177215,0.225,0.546099,0.392405,0.891892,0.216216


# **Part 3: Creating and Testing Model**

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [30]:
def monte_carlo(n):
    accuracy = []
    for i in range(n):
        # Split the data into training and test sets
        X_train, X_test, y_train, y_test = train_test_split(df[predictors], df['Home_Winner'], test_size=0.2)

        # Train a logistic regression model on the training data
        model = LogisticRegression()
        # Fit the model to our training data
        model.fit(X_train, y_train)

        # Predict the winners for the test data
        y_pred = model.predict(X_test)

        # Evaluate the accuracy of the model on the test data
        accuracy.append(accuracy_score(y_test, y_pred))

    # Get the average accuracy
    score = sum(accuracy) / len(accuracy)
    return score

In [31]:
score = monte_carlo(1000)
print(f"Accuracy: {score}")

Accuracy: 0.7037500000000003


# **Part 4: Predicting Finals**

In [33]:
# Remove the rows where the Aces and Liberty play against each other
non_aces_liberty_game = df
non_aces_liberty_game = non_aces_liberty_game.drop(non_aces_liberty_game[(non_aces_liberty_game['Home/Neutral'] == 'Las Vegas Aces') & (non_aces_liberty_game['Visitor/Neutral'] == 'New York Liberty')].index)
non_aces_liberty_game = non_aces_liberty_game.drop(non_aces_liberty_game[(non_aces_liberty_game['Home/Neutral'] == 'New York Liberty') & (non_aces_liberty_game['Visitor/Neutral'] == 'Las Vegas Aces')].index)
non_aces_liberty_game[(non_aces_liberty_game['Home/Neutral'] == 'Las Vegas Aces') & (non_aces_liberty_game['Visitor/Neutral'] == 'New York Liberty')]

,Visitor/Neutral,PTS,Home/Neutral,PTS.1,Age_x,W_x,L_x,PW_x,PL_x,MOV_x,...,TS%_y,eFG%_y,TOV%_y,ORB%_y,FT/FGA_y,eFG%.1_y,TOV%.1_y,DRB%_y,FT/FGA.1_y,Home_Winner


In [34]:
# Get a game where the Aces are home and Liberty is away
final_matchup = df[(df['Home/Neutral'] == 'Las Vegas Aces') & (df['Visitor/Neutral'] == 'New York Liberty')][:1]
# Show the predictors we will use
final_matchup[predictors]

,ORB%_x,eFG%.1_x,TOV%.1_x,L_y,TS%_y,eFG%_y,ORB%_y,FT/FGA_y,eFG%.1_y,TOV%.1_y
144,0.503546,0.0,0.0,0.0,1.0,1.0,0.014184,0.797468,0.108108,0.351351


In [35]:
# Predict the winner of the final matchup
model = LogisticRegression()
model.fit(non_aces_liberty_game[predictors], non_aces_liberty_game['Home_Winner'])

LogisticRegression()

In [36]:
# Predict the outcome of the final_matchup
y_pred = model.predict(final_matchup[predictors])
print(f"Prediction: {y_pred[0]}")

Prediction: 1.0
